In [1]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta, timezone
import time
import nvda_config
import twelvedata as td
from twelvedata import TDClient


In [2]:
# set up login
Cpassword = nvda_config.mysql_password
td = TDClient(apikey=nvda_config.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'


In [3]:
# Estab;ish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()




In [4]:


# Twelve Data API parameters
symbol = "NVDA"
interval = "1min"
outputsize = 5000 

# Initialize a counter for the number of requests
request_count = 0

while request_count < 390:
    # Fetch time series data and indicators
    price_pdf = td.price(
        symbol=symbol
    )
    # Convert to pandas Dataframes
    price_json = price_pdf.as_json()
    price_df = pd.DataFrame(price_json, index=["price"])

    # Prepare for database insertion
    time_series_data = []

    # Process and align data
    for index, row in price_df.iterrows():
        # Generate the current datetime string
        datetime_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        time_series_data.append(
            (
                symbol,
                datetime_str,
                row["price"]
            )
        )

    # Insert tick data into MySQL
    sql_time_series = """
    INSERT INTO nvda_new (symbol, datetime, price)
    VALUES (%s, %s, %s)
    """
    mycursor.executemany(sql_time_series, time_series_data)
    mydb.commit()

    # Increment the request counter
    request_count += 1
    print("Requests elapsed: ", request_count, " at time:", datetime.now())
    # Sleep for a short period before fetching the next data
    time.sleep(60)  # Adjust the sleep time as needed

print("Program completed after 390 requests.")


Requests elapsed:  1  at time: 2024-08-07 06:39:15.096636
Requests elapsed:  2  at time: 2024-08-07 06:40:15.333265
Requests elapsed:  3  at time: 2024-08-07 06:41:15.564956
Requests elapsed:  4  at time: 2024-08-07 06:42:15.810862
Requests elapsed:  5  at time: 2024-08-07 06:43:16.074925
Requests elapsed:  6  at time: 2024-08-07 06:44:16.326217
Requests elapsed:  7  at time: 2024-08-07 06:45:16.635438
Requests elapsed:  8  at time: 2024-08-07 06:46:16.869411
Requests elapsed:  9  at time: 2024-08-07 06:47:17.146361
Requests elapsed:  10  at time: 2024-08-07 06:48:17.400895
Requests elapsed:  11  at time: 2024-08-07 06:49:17.657995
Requests elapsed:  12  at time: 2024-08-07 06:50:17.898012
Requests elapsed:  13  at time: 2024-08-07 06:51:18.144186
Requests elapsed:  14  at time: 2024-08-07 06:52:18.406276
Requests elapsed:  15  at time: 2024-08-07 06:53:18.642544
Requests elapsed:  16  at time: 2024-08-07 06:54:18.895384
Requests elapsed:  17  at time: 2024-08-07 06:55:19.128995
Reques